In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Data Generation

In [2]:
classes =[0,1]
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0 #normalize the data

x_train_filter_01 = np.where((y_train == classes[0]) | (y_train == classes[1]))
x_test_filter_01 = np.where((y_test == classes[0]) | (y_test == classes[1]))

x_train_01, x_test_01 = x_train[x_train_filter_01], x_test[x_test_filter_01]
y_train_01, y_test_01 = y_train[x_train_filter_01], y_test[x_test_filter_01]
x_train_01 = tf.image.resize(x_train_01[:], (784, 1)).numpy()
x_test_01 = tf.image.resize(x_test_01[:], (784, 1)).numpy()
x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01)

pca = PCA(8)
x_train_01 = pca.fit_transform(x_train_01)
x_test_01 = pca.transform(x_test_01)

In [3]:
#Reshape data dimension suitable to CNN input shape
x = np.array([np.reshape(x_train_01[i], (8,1,1)) for i in range(len(x_train_01))])
xt = np.array([np.reshape(x_test_01[i], (8,1,1)) for i in range(len(x_test_01))])

In [4]:
x.shape #Check: should be (12665, 8, 1, 1)
xt.shape #Check: should be (2115, 8, 1, 1)

(2115, 8, 1, 1)

In [5]:
learning_rate = 0.01
iteration = 300
batch_size = 25


optimizer = tf.keras.optimizers.Adam(
    learning_rate=learning_rate, beta_1=0.9, beta_2=0.99, epsilon=1e-08, amsgrad=False,
    name='Adam')

accuracy = []


for _ in range(5):    
    #Make CNN
    model = models.Sequential()
    model.add(layers.Conv2D(3, (2, 1), activation = 'relu', input_shape=(8, 1, 1)))
    model.add(layers.MaxPooling2D((2, 1)))
    model.add(layers.Conv2D(3, (2, 1), activation='relu'))
    model.add(layers.MaxPooling2D((2, 1)))
    model.add(layers.Flatten())
    #model.add(layers.Dense(4, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))

    #Compile CNN
    model.compile(optimizer=optimizer,
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

    for i in range(iteration):

        #Generate Random Batch
        batch_index = np.random.randint(0, 12665, (batch_size,))
        X_batch = np.array([x[i] for i in batch_index])
        Y_batch = np.array([y_train_01[i] for i in batch_index])

        #Train CNN
        model.fit(X_batch, Y_batch, batch_size = batch_size, epochs=1)

    test_loss, test_acc = model.evaluate(xt, y_test_01, verbose=2)
    
    accuracy.append(test_acc)
    
    #Clear CNN Model
    tf.keras.backend.clear_session()

print(accuracy)

1/1 [==============================] - 0s 2ms/step - loss: 0.1264 - accuracy: 0.9600


1/1 [==============================] - 0s 3ms/step - loss: 0.0688 - accuracy: 0.9600


1/1 [==============================] - 0s 3ms/step - loss: 0.0249 - accuracy: 1.0000


1/1 [==============================] - 0s 3ms/step - loss: 0.0114 - accuracy: 1.0000
67/67 - 0s - loss: 0.0642 - accuracy: 0.9801
1/1 [==============================] - 0s 3ms/step - loss: 0.1736 - accuracy: 0.9600


1/1 [==============================] - 0s 2ms/step - loss: 0.2204 - accuracy: 0.9600


...

1/1 [==============================] - 0s 2ms/step - loss: 0.3506 - accuracy: 0.8800


1/1 [==============================] - 0s 2ms/step - loss: 0.4413 - accuracy: 0.9200
67/67 - 0s - loss: 0.0761 - accuracy: 0.9783
1/1 [==============================] - 0s 2ms/step - loss: 0.3298 - accuracy: 0.8800


1/1 [==============================] - 0s 2ms/step - loss: 0.3423 - accuracy: 0.8800


1/1 [==============================] - 0s 2ms/step - loss: 0.1350 - accuracy: 0.9600


1/1 [==============================] - 0s 3ms/step - loss: 0.1056 - accuracy: 1.0000
67/67 - 0s - loss: 0.1963 - accuracy: 0.9248
[0.980141818523407, 0.9815602898597717, 0.9338061213493347, 0.9782505631446838, 0.9248226881027222]


In [6]:
model = models.Sequential()
model.add(layers.Conv2D(3, (2, 1), activation = 'relu', input_shape=(8, 1, 1)))
model.add(layers.MaxPooling2D((2, 1)))
model.add(layers.Conv2D(3, (2, 1), activation='relu'))
model.add(layers.MaxPooling2D((2, 1)))
model.add(layers.Flatten())
#model.add(layers.Dense(2, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 7, 1, 3)           9         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 1, 3)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 1, 3)           21        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 3)           0         
_________________________________________________________________
flatten (Flatten)            (None, 3)                 0         
_________________________________________________________________
dense (Dense)                (None, 2)                 8         
Total params: 38
Trainable params: 38
Non-trainable params: 0
____________________________________________________________

In [7]:
tf.keras.backend.clear_session()